## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [1]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "the Memorial Tournament presented by Workday"
tournament_date = "6/1/2025"  # Ending date of tournament
old_course = "Muirfield Village Golf Club"
tournament_id = "R2025023"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "RBC Canadian Open"
new_ending_date = "6/8/2025"
new_course = "TPC Toronto at Osprey Valley"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [2]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025023 (the Memorial Tournament presented by Workday), year: 20250
✅ 72 new rows added for 'the Memorial Tournament presented by Workday'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
58,2025,2025-06-01,R2025023,the Memorial Tournament presented by Workday,Muirfield Village Golf Club,Scottie Scheffler,1,1,-2,-2,-4,-2,"$4,000,000.00",700.000
7,2025,2025-06-01,R2025023,the Memorial Tournament presented by Workday,Muirfield Village Golf Club,Ben Griffin,2,2,-7,E,E,+1,"$2,200,000.00",400.000
59,2025,2025-06-01,R2025023,the Memorial Tournament presented by Workday,Muirfield Village Golf Club,Sepp Straka,3,3,+2,+1,-6,-2,"$1,400,000.00",350.000
48,2025,2025-06-01,R2025023,the Memorial Tournament presented by Workday,Muirfield Village Golf Club,Nick Taylor,4,4,-3,-4,+2,+1,"$1,000,000.00",325.000
42,2025,2025-06-01,R2025023,the Memorial Tournament presented by Workday,Muirfield Village Golf Club,Maverick McNealy,T5,5,+3,E,-3,-2,"$800,000.00",287.500


#### Stats

In [3]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 999 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,590,.1991,2025
1,Aaron Baddeley,167.0,-0.965,178.0,-1.366,134.0,-0.203,3.0,0.604,161.0,...,288.9,122.0,56.64%,169.0,60.61%,86.0,60.68%,434,.3029,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,345,.4103,2025
3,Aaron Rai,26.0,0.732,27.0,0.349,33.0,0.403,104.0,-0.019,128.0,...,286.6,1.0,73.43%,15.0,69.21%,128.0,58.27%,30,2.6863,2025
4,Aaron Wilkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,786,.1242,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [30]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

ℹ️ No odds rows required name cleanup.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [31]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

ℹ️ Historical odds for season 2023 already exist — no new rows added.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [32]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [33]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [34]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [4]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 7 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
466,2016,Glen Abbey GC,032,RBC Canadian Open,2016-07-24
310,2017,Glen Abbey GC,032,RBC Canadian Open,2017-07-30
156,2018,Glen Abbey GC,032,RBC Canadian Open,2018-07-29
0,2019,Hamilton Golf & Country Club,032,RBC Canadian Open,2019-06-09
776,2022,St. George's G&CC,032,RBC Canadian Open,2022-06-12
930,2023,Oakdale Golf & Country Club,R2023032,RBC Canadian Open,2023-06-11
1086,2024,Hamilton Golf & Country Club,R2024032,RBC Canadian Open,2024-06-02


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [5]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-07-24 — RBC Canadian Open (489 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,16,9,364.5,56.2,22.78,0,2016-07-24,RBC Canadian Open
1,Aaron Wise,1,0,0.0,0.0,0.00,0,2016-07-24,RBC Canadian Open
2,Abraham Ancer,9,2,43.0,22.2,4.78,2,2016-07-24,RBC Canadian Open



📆 2017-07-30 — RBC Canadian Open (508 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,15,7,254.14,46.7,16.94,1,2017-07-30,RBC Canadian Open
1,Aaron Rai,1,0,0.00,0.0,0.00,0,2017-07-30,RBC Canadian Open
2,Aaron Wise,5,4,0.00,80.0,0.00,4,2017-07-30,RBC Canadian Open



📆 2018-07-29 — RBC Canadian Open (529 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,5,2,56.96,40.0,11.39,0,2022-06-12,RBC Canadian Open
1,Aaron Beverly,1,0,0.00,0.0,0.00,0,2022-06-12,RBC Canadian Open
2,Aaron Jarvis,1,0,0.00,0.0,0.00,0,2022-06-12,RBC Canadian Open



📆 2023-06-11 — RBC Canadian Open (489 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,11,342.286,78.6,24.45,5,2023-06-11,RBC Canadian Open
1,Aaron Beverly,1,1,0.000,100.0,0.00,1,2023-06-11,RBC Canadian Open
2,Aaron Jarvis,1,0,0.000,0.0,0.00,0,2023-06-11,RBC Canadian Open



📆 2024-06-02 — RBC Canadian Open (449 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,13,9,181.975,69.2,14.00,4,2024-06-02,RBC Canadian Open
1,Aaron Rai,14,11,391.007,78.6,27.93,5,2024-06-02,RBC Canadian Open
2,Adam Hadwin,14,11,679.292,78.6,48.52,5,2024-06-02,RBC Canadian Open


### Recent Form

In [6]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-07-24 — RBC Canadian Open (489 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Lee McCoy,1,4.0,5.77,2016-07-24,RBC Canadian Open
1,Tyrrell Hatton,1,5.0,7.21,2016-07-24,RBC Canadian Open
2,Dustin Johnson,12,11.2,4.37,2016-07-24,RBC Canadian Open



📆 2017-07-30 — RBC Canadian Open (508 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Matthew Southgate,1,6.0,8.66,2017-07-30,RBC Canadian Open
1,Keith Mitchell,1,11.0,15.87,2017-07-30,RBC Canadian Open
2,Oscar Fraustro,1,13.0,18.76,2017-07-30,RBC Canadian Open



📆 2018-07-29 — RBC Canadian Open (529 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Taylor Montgomery,1,11.0,15.87,2022-06-12,RBC Canadian Open
1,Haotong Li,1,12.0,17.31,2022-06-12,RBC Canadian Open
2,Martin Contini,1,16.0,23.08,2022-06-12,RBC Canadian Open



📆 2023-06-11 — RBC Canadian Open (489 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Joohyung Kim,1,1.0,1.44,2023-06-11,RBC Canadian Open
1,Brooks Koepka,2,1.5,1.37,2023-06-11,RBC Canadian Open
2,Scottie Scheffler,16,8.1,2.86,2023-06-11,RBC Canadian Open



📆 2024-06-02 — RBC Canadian Open (449 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Bryson DeChambeau,2,4.0,3.64,2024-06-02,RBC Canadian Open
1,Scottie Scheffler,13,4.5,1.71,2024-06-02,RBC Canadian Open
2,Dean Burmester,1,12.0,17.31,2024-06-02,RBC Canadian Open


### Course History

In [7]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))

## Training Dataset

In [8]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
0,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Abraham Ancer,CUT,90,144.0,-0.307,111.0,-0.024,161.0,-0.340,77.0,0.057,172.0,-0.421,154.0,18.63,120.0,3.08,136.0,4.07,183.0,4.80,131.0,205.0,178.0,276.4,27.0,65.42,174.0,61.81,131.0,56.67,619.0,0.25,NaN,22.2,43.00,4.78,2.0,80.1,34.79,0
1,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Adam Cornelson,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,688.0,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Adam Hadwin,T49,49,111.0,-0.011,55.0,0.223,121.0,-0.055,132.0,-0.081,11.0,0.530,22.0,21.74,16.0,3.01,48.0,4.03,68.0,4.66,35.0,141.0,92.0,290.1,49.0,63.62,83.0,66.07,97.0,58.27,193.0,0.91,80.0,72.2,457.67,25.43,6.0,50.2,17.05,0
3,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Alex Cejka,T5,5,39.0,0.631,148.0,-0.292,12.0,0.626,20.0,0.297,90.0,0.010,17.0,21.89,3.0,2.98,14.0,4.01,10.0,4.57,78.0,177.0,155.0,281.5,22.0,65.89,39.0,67.99,3.0,65.03,130.0,1.22,NaN,60.0,503.67,33.58,1.0,52.8,19.04,1
4,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Andres Gonzales,CUT,90,160.0,-0.469,141.0,-0.222,134.0,-0.112,151.0,-0.135,94.0,0.007,91.0,20.01,78.0,3.06,162.0,4.09,80.0,4.67,145.0,224.0,59.0,294.7,165.0,54.55,81.0,66.17,148.0,55.79,270.0,0.66,NaN,35.3,52.80,3.11,0.0,80.2,27.75,0
5,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Andrew Landry,CUT,90,172.0,-0.694,11.0,0.594,181.0,-0.892,179.0,-0.396,4.0,0.654,160.0,18.48,136.0,3.09,34.0,4.02,169.0,4.76,5.0,85.0,59.0,294.7,26.0,65.55,117.0,64.63,28.0,61.86,482.0,0.37,NaN,44.4,79.29,8.81,4.0,73.4,31.88,0
6,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Andrew Loupe,CUT,90,150.0,-0.395,105.0,-0.005,154.0,-0.281,158.0,-0.172,38.0,0.280,12.0,22.74,78.0,3.06,177.0,4.11,13.0,4.58,101.0,189.0,5.0,309.3,184.0,45.56,148.0,63.41,175.0,52.83,194.0,0.91,NaN,52.9,420.25,24.72,0.0,60.7,21.00,0
7,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Ben Crane,T9,9,115.0,-0.028,117.0,-0.063,99.0,0.101,49.0,0.173,46.0,0.230,128.0,19.23,78.0,3.06,48.0,4.03,100.0,4.69,138.0,216.0,162.0,280.3,54.0,63.18,143.0,63.6,31.0,61.65,371.0,0.50,NaN,53.3,287.00,19.13,2.0,59.3,21.39,1
8,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Ben Martin,CUT,90,126.0,-0.117,79.0,0.079,101.0,0.081,169.0,-0.277,70.0,0.140,29.0,21.53,146.0,3.10,97.0,4.05,28.0,4.61,71.0,174.0,108.0,288.8,66.0,62.14,64.0,66.73,160.0,54.84,123.0,1.27,60.0,71.4,316.55,22.61,2.0,52.4,19.35,0
9,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Ben Taylor,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [9]:
# Play with different years to see if I notice anything wrong
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
930,2024,2024-06-02,RBC Canadian Open,Hamilton Golf & Country Club,Aaron Baddeley,CUT,90,167.0,-0.982,175.0,-1.009,153.0,-0.375,7.0,0.402,19.0,0.470,155.0,20.20%,11.0,2.99,141.0,4.05,158.0,4.67,175.0,316.0,171.0,284.8,145.0,56.21%,173.0,62.17%,7.0,65.97%,299.0,0.4698,250.0,69.2,181.975,14.00,4.0,54.8,20.76,0
931,2024,2024-06-02,RBC Canadian Open,Hamilton Golf & Country Club,Aaron Cockerill,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,251.0,0.5632,300.0,NaN,NaN,NaN,NaN,NaN,NaN,0
932,2024,2024-06-02,RBC Canadian Open,Hamilton Golf & Country Club,Aaron Rai,T14,14,9.0,1.131,36.0,0.303,8.0,0.676,56.0,0.152,83.0,0.054,99.0,22.10%,38.0,3.01,10.0,3.97,44.0,4.54,53.0,153.0,152.0,293.8,1.0,72.02%,7.0,71.58%,33.0,63.18%,21.0,3.2294,40.0,78.6,391.007,27.93,5.0,43.1,15.92,1
933,2024,2024-06-02,RBC Canadian Open,Hamilton Golf & Country Club,Adam Hadwin,CUT,90,112.0,-0.036,101.0,0.016,113.0,-0.057,102.0,0.005,82.0,0.058,63.0,23.00%,52.0,3.02,131.0,4.04,116.0,4.61,123.0,198.0,124.0,297.9,74.0,61.57%,152.0,64.30%,151.0,55.81%,56.0,2.0027,50.0,78.6,679.292,48.52,5.0,42.8,15.80,0
934,2024,2024-06-02,RBC Canadian Open,Hamilton Golf & Country Club,Adam Long,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,548.0,0.2354,500.0,50.0,125.756,12.58,0.0,62.0,25.86,0


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [11]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
0,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Abraham Ancer,CUT,90,144.0,-0.307,111.0,-0.024,161.0,-0.340,77.0,0.057,172.0,-0.421,154.0,18.63,120.0,3.08,136.0,4.07,183.0,4.80,131.0,205.0,178.0,276.4,27.0,65.42,174.0,61.81,131.0,56.67,619.0,0.25,1000.0,22.2,43.00,4.78,2.0,80.1,34.79,0
1,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Adam Cornelson,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,688.0,0.22,1000.0,NaN,0.00,NaN,NaN,90.0,NaN,0
2,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Adam Hadwin,T49,49,111.0,-0.011,55.0,0.223,121.0,-0.055,132.0,-0.081,11.0,0.530,22.0,21.74,16.0,3.01,48.0,4.03,68.0,4.66,35.0,141.0,92.0,290.1,49.0,63.62,83.0,66.07,97.0,58.27,193.0,0.91,80.0,72.2,457.67,25.43,6.0,50.2,17.05,0
3,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Alex Cejka,T5,5,39.0,0.631,148.0,-0.292,12.0,0.626,20.0,0.297,90.0,0.010,17.0,21.89,3.0,2.98,14.0,4.01,10.0,4.57,78.0,177.0,155.0,281.5,22.0,65.89,39.0,67.99,3.0,65.03,130.0,1.22,1000.0,60.0,503.67,33.58,1.0,52.8,19.04,1
4,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Andres Gonzales,CUT,90,160.0,-0.469,141.0,-0.222,134.0,-0.112,151.0,-0.135,94.0,0.007,91.0,20.01,78.0,3.06,162.0,4.09,80.0,4.67,145.0,224.0,59.0,294.7,165.0,54.55,81.0,66.17,148.0,55.79,270.0,0.66,1000.0,35.3,52.80,3.11,0.0,80.2,27.75,0


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [12]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
0,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Abraham Ancer,CUT,90,144.000000,-0.307000,111.000000,-0.024000,161.000000,-0.340000,77.000000,0.057000,172.000000,-0.421000,154.000000,18.630000,120.00000,3.080000,136.000000,4.070000,183.000000,4.800000,131.000000,205.000000,178.000000,276.400000,27.000000,65.420000,174.000000,61.810000,131.00000,56.670000,619.0,0.25,1000.0,22.200000,43.00,4.780000,2.000000,80.1,34.790000,0
1,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Adam Cornelson,CUT,90,106.746305,-0.022897,102.768473,-0.017188,102.812808,0.001842,102.682266,-0.007384,99.294335,0.001728,102.245074,20.810419,91.99877,3.057294,94.742927,4.035683,95.725707,4.640578,102.097291,196.006158,100.491379,295.768227,95.514778,60.619581,94.836408,66.248954,99.04064,58.804643,688.0,0.22,1000.0,51.549228,0.00,18.205757,1.290422,90.0,27.701205,0
2,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Adam Hadwin,T49,49,111.000000,-0.011000,55.000000,0.223000,121.000000,-0.055000,132.000000,-0.081000,11.000000,0.530000,22.000000,21.740000,16.00000,3.010000,48.000000,4.030000,68.000000,4.660000,35.000000,141.000000,92.000000,290.100000,49.000000,63.620000,83.000000,66.070000,97.00000,58.270000,193.0,0.91,80.0,72.200000,457.67,25.430000,6.000000,50.2,17.050000,0
3,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Alex Cejka,T5,5,39.000000,0.631000,148.000000,-0.292000,12.000000,0.626000,20.000000,0.297000,90.000000,0.010000,17.000000,21.890000,3.00000,2.980000,14.000000,4.010000,10.000000,4.570000,78.000000,177.000000,155.000000,281.500000,22.000000,65.890000,39.000000,67.990000,3.00000,65.030000,130.0,1.22,1000.0,60.000000,503.67,33.580000,1.000000,52.8,19.040000,1
4,2016,2016-07-24,RBC Canadian Open,Glen Abbey GC,Andres Gonzales,CUT,90,160.000000,-0.469000,141.000000,-0.222000,134.000000,-0.112000,151.000000,-0.135000,94.000000,0.007000,91.000000,20.010000,78.00000,3.060000,162.000000,4.090000,80.000000,4.670000,145.000000,224.000000,59.000000,294.700000,165.000000,54.550000,81.000000,66.170000,148.00000,55.790000,270.0,0.66,1000.0,35.300000,52.80,3.110000,0.000000,80.2,27.750000,0


In [13]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [14]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [ ]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [16]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk.head(10)

,PLAYER,SALARY
0,Rory McIlroy,11900
1,Ludvig Aberg,10500
2,Corey Conners,10100
3,Shane Lowry,9900
4,Robert Macintyre,9800
5,Taylor Pendrith,9600
6,Sungjae Im,9400
7,Sam Burns,9300
8,Harry Hall,9200
9,Keith Mitchell,9100


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [17]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,RBC Canadian Open,Rory McIlroy,9/2,4.5
2,2025,RBC Canadian Open,Ludvig Aberg,14/1,14.0
3,2025,RBC Canadian Open,Corey Conners,18/1,18.0
4,2025,RBC Canadian Open,Shane Lowry,20/1,20.0
5,2025,RBC Canadian Open,Taylor Pendrith,20/1,20.0
6,2025,RBC Canadian Open,Robert MacIntyre,25/1,25.0
7,2025,RBC Canadian Open,Sam Burns,25/1,25.0
8,2025,RBC Canadian Open,Sungjae Im,30/1,30.0
9,2025,RBC Canadian Open,Luke Clanton,40/1,40.0
10,2025,RBC Canadian Open,Mackenzie Hughes,40/1,40.0


## Cut Percentage and FedEx Points

In [18]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,15,7,55.951,46.7,3.73,0,2025-06-08,RBC Canadian Open
1,Aaron Rai,15,11,577.480,73.3,38.50,0,2025-06-08,RBC Canadian Open
2,Adam Hadwin,16,10,177.013,62.5,11.06,1,2025-06-08,RBC Canadian Open
3,Adam Long,1,0,0.000,0.0,0.00,0,2025-06-08,RBC Canadian Open
4,Adam Schenk,18,7,253.954,38.9,14.11,0,2025-06-08,RBC Canadian Open
5,Adam Scott,11,9,264.810,81.8,24.07,4,2025-06-08,RBC Canadian Open
6,Adam Svensson,19,11,173.158,57.9,9.11,1,2025-06-08,RBC Canadian Open
7,Adrien Dumont de Chassart,8,3,200.500,37.5,25.06,0,2025-06-08,RBC Canadian Open
8,Akshay Bhatia,15,12,800.240,80.0,53.35,2,2025-06-08,RBC Canadian Open
9,Aldrich Potgieter,11,4,441.000,36.4,40.09,1,2025-06-08,RBC Canadian Open


## Recent Form

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Bryson DeChambeau,2,3.5,3.19,2025-06-08,RBC Canadian Open
1,Scottie Scheffler,13,6.9,2.61,2025-06-08,RBC Canadian Open
2,Jon Rahm,2,11.0,10.01,2025-06-08,RBC Canadian Open
3,Rory McIlroy,8,11.8,5.37,2025-06-08,RBC Canadian Open
4,John Keefer,1,13.0,18.76,2025-06-08,RBC Canadian Open
5,Bubba Watson,1,14.0,20.20,2025-06-08,RBC Canadian Open
6,Taisei Shimizu,1,16.0,23.08,2025-06-08,RBC Canadian Open
7,Ian Gilligan,1,16.0,23.08,2025-06-08,RBC Canadian Open
8,Joaquin Niemann,2,18.5,16.84,2025-06-08,RBC Canadian Open
9,Tommy Fleetwood,12,18.6,7.25,2025-06-08,RBC Canadian Open


## Course History

In [20]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


""


## Merged Dataframe

In [26]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"]
)

this_week.head()

,SEASON,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,SALARY
0,2025.0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,590.0,0.1991,600.0,NaN,NaN,NaN,NaN,NaN,NaN,6400
1,2025.0,Aaron Baddeley,167.0,-0.965,178.0,-1.366,134.0,-0.203,3.0,0.604,161.0,-0.470,177.0,18.01%,107.0,3.08,152.0,4.07,178.0,4.78,175.0,288.0,166.0,288.9,122.0,56.64%,169.0,60.61%,86.0,60.68%,434.0,0.3029,1000.0,46.7,55.951,3.73,0.0,74.9,27.01,6200
2,NaN,Aaron Wise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6500
3,2025.0,Adam Hadwin,159.0,-0.630,145.0,-0.264,162.0,-0.505,60.0,0.139,117.0,-0.071,72.0,22.10%,37.0,3.03,152.0,4.07,151.0,4.68,120.0,200.0,112.0,298.1,88.0,59.15%,160.0,61.70%,167.0,54.32%,105.0,1.2520,150.0,62.5,177.013,11.06,1.0,63.2,22.31,6900
4,2025.0,Adam Schenk,104.0,-0.061,78.0,0.117,108.0,-0.067,125.0,-0.111,124.0,-0.117,99.0,21.32%,78.0,3.06,135.0,4.06,40.0,4.56,133.0,210.0,44.0,306.0,166.0,51.75%,164.0,61.26%,74.0,61.24%,149.0,0.8884,250.0,38.9,253.954,14.11,0.0,67.6,22.96,6700


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [27]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# === Normalize & Assign Odds for Poor or Unknown Players ===
this_week["VEGAS_ODDS"] = this_week["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === Normalize & Assign OWGR & OWGR_RANK for Unranked Players ===
# OWGR (raw) — assign worst actual rank in dataset
if "OWGR" in this_week.columns:
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).astype(float).clip(upper=1000)

# OWGR_RANK — if present, use 1000 for missing (very poor)
if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = this_week["OWGR_RANK"].fillna(1000).clip(upper=1000)

# === Assign RECENT FORM Score for Players Who Haven’t Played Recently ===
this_week["RECENT_FORM"] = this_week["RECENT_FORM"].fillna(90)

# === Assign 0 to FEDEX_CUP_POINTS Where No Data ===
this_week["FEDEX_CUP_POINTS"] = this_week["FEDEX_CUP_POINTS"].fillna(0)

# === Course History NaNs: Fill with Mean (not 0 — 0 implies bad, not missing) ===
if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(this_week["COURSE_HISTORY"].mean())

# === Fill Remaining NaNs in Numeric Columns with Column Mean ===
numeric_cols = this_week.select_dtypes(include=["float64", "int64"])
numeric_filled = numeric_cols.fillna(numeric_columns.mean())
this_week.update(numeric_filled)



In [28]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [29]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

## Correlations
**Last 4 Years**

In [30]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [31]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [32]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 37 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


📊 Random Forest (All Features)
Cross-validation scores: [0.97216441 0.94068678 0.94403977 0.9663834  0.94771072]
Average ROC AUC score: 0.95


Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [35]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,Rory McIlroy,11900,0.931149,11.8,4.5,100.000000,2.0,2635.786,3.000000,1.000000,39.000000,41.000000,6.000000,9.000000,14.00000,4.000000,9.000000,83.000000,3.000000,170.000000,50.000000,10.00000
1,Shane Lowry,9900,0.885326,29.3,20.0,83.300000,12.0,1318.139,2.000000,49.000000,3.000000,22.000000,96.000000,76.000000,14.00000,39.000000,138.000000,80.000000,147.000000,25.000000,48.000000,26.00000
2,Ryo Hisatsune,7600,0.857285,46.9,80.0,68.400000,84.0,560.719,36.000000,88.000000,45.000000,34.000000,74.000000,130.000000,8.00000,73.000000,74.000000,164.000000,130.000000,105.000000,51.000000,16.00000
3,Nick Taylor,8800,0.792983,43.7,50.0,73.700000,32.0,1280.032,39.000000,123.000000,13.000000,69.000000,72.000000,86.000000,37.00000,11.000000,124.000000,99.000000,164.000000,20.000000,12.000000,35.00000
4,Harry Hall,9200,0.732540,34.9,40.0,83.300000,85.0,716.663,67.000000,111.000000,105.000000,19.000000,4.000000,7.000000,37.00000,4.000000,50.000000,113.000000,88.000000,104.000000,81.000000,29.00000
5,Mackenzie Hughes,8900,0.668439,48.4,40.0,68.400000,50.0,813.981,90.000000,138.000000,117.000000,21.000000,88.000000,129.000000,51.00000,112.000000,108.000000,145.000000,95.000000,124.000000,111.000000,60.00000
6,Alex Noren,8500,0.656932,35.7,50.0,100.000000,73.0,104.000,106.746305,102.768473,102.812808,102.682266,99.294335,102.245074,91.99877,94.742927,95.725707,102.097291,100.491379,95.514778,94.836408,99.04064
7,Ryan Fox,7700,0.546987,48.5,70.0,78.600000,75.0,300.750,40.000000,86.000000,35.000000,72.000000,77.000000,43.000000,141.00000,56.000000,68.000000,107.000000,56.000000,131.000000,119.000000,150.00000
8,Alex Smalley,7600,0.506762,47.5,60.0,65.000000,117.0,584.188,9.000000,13.000000,63.000000,10.000000,40.000000,31.000000,107.00000,3.000000,60.000000,2.000000,50.000000,30.000000,34.000000,14.00000
9,Corey Conners,10100,0.476064,27.5,18.0,93.300000,21.0,1438.934,30.000000,9.000000,58.000000,111.000000,61.000000,64.000000,96.00000,23.000000,74.000000,65.000000,151.000000,13.000000,18.000000,70.00000


### Save to CSV

In [38]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

# Export to CSV
filename = f"data/{tournament_config['new']['name'].replace(' ', '_')}_{tournament_config['new']['ending_date'].date()}.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/RBC_Canadian_Open_2025-06-08.csv
